In [1]:
import WesBotTWO as wb
import HandOddsCalcWes as hoc

import texasholdem as th
import texasholdem.evaluator as eval
import numpy as np
import pandas as pd
import random
from tqdm import trange

In [2]:
# smort_decision(our_bot:PokerBot, game:th.TexasHoldEm)
# actually make a goddamn decision using my scuffed bot thingy, and record
# all the necessary information for Shro' to use in his thing
# INPUTS:
#   our_bot: a PokerBot object, supposed to be intelligent or something
#   game:    a TexasHoldEm game object that we're playing
# OUTPUT:
#   a list containing:
#     [hand_phase:str,
#      my_hand:list of th.Card,
#      board:list of th.Card,
#      EV:float,
#      my_chips_betting:int,
#      their_chips_betting:int,
#      my_decision:Decision]
# SIDE EFFECT:
#   makes the decision in the TexasHoldEm game object passed in
def smort_decision(our_bot:wb.PokerBot, game:th.TexasHoldEm):
  # get information regarding our hand, the game board, our win prob,
  # their chips bet, and our EV
  win_prob =  hoc.estimate_win_prob(game, game.current_player, 2, 1000)
  loss_prob = 1 - win_prob
  my_chips_betting = game.player_bet_amount(0)
  their_chips_betting = game.player_bet_amount(1)
  EV = win_prob*their_chips_betting - loss_prob*(my_chips_betting + \
                                                 game.chips_to_call(0))
  # get features for Shro'
  my_decision = our_bot.make_decision(EV, game)
  my_hand = game.get_hand(game.current_player)
  board = []
  for card in game.board:
    board.append(card)
  # record hand phase
  hand_phase = game.hand_phase.name
  # actually make the decision
  if (my_decision.type == "RAISE"):
    if (game.validate_move(action = th.ActionType.RAISE,\
                           value = my_decision.size)):
      game.take_action(th.ActionType.RAISE, my_decision.size)
    else:
      game.take_action(th.ActionType.ALL_IN)
  elif (my_decision.type == "CALL/CHECK"):
    if (game.validate_move(action = th.ActionType.CALL)):
      game.take_action(th.ActionType.CALL)
    else:
      game.take_action(th.ActionType.CHECK)
  elif (my_decision.type == "FOLD"):
    game.take_action(th.ActionType.FOLD)
  else:
    game.take_action(th.ActionType.ALL_IN)
  output = [hand_phase, my_hand, board, EV, my_chips_betting, their_chips_betting, my_decision]
  return output

# baby_decision(game:th.TexasHoldEm)
# make a random decision for the opponent, just to get someone to play against
# INPUTS:
#   game: a th.TexasHoldEm object that we're playing
# SIDE EFFECT:
#   makes the random decision in the game object passed in
def baby_decision(game:th.TexasHoldEm):
  # make random decision
  if (np.random.rand() > 0.5):
    # baby gonna raise
    min_raise = int(game.get_available_moves().raise_range.start)
    max_raise = int(np.min([game.players[0].chips,
                            game.get_available_moves().raise_range.stop]))
    # make sure we only raise by some reasonable amount
    max_raise = int((3*min_raise + max_raise) // 4)
    if (max_raise - min_raise > 20):
      max_raise = min_raise + 20
    decision = \
      wb.Decision("RAISE", int(np.random.uniform(min_raise, max_raise)))
  else:
    # baby gonna call/check
    decision = wb.Decision("CALL/CHECK")
  # ensure validity of decision and actually make decision
  if (decision.type == "RAISE" and
      game.validate_move(action = th.ActionType.RAISE, value = decision.size)):
    game.take_action(th.ActionType.RAISE, decision.size)
    return -1
  elif (game.validate_move(action = th.ActionType.CALL)):
    game.take_action(th.ActionType.CALL)
    return -1
  elif (game.validate_move(action = th.ActionType.CHECK)):
    game.take_action(th.ActionType.CHECK)
    return -1
  game.take_action(th.ActionType.FOLD)
  return 0
  
      

In [3]:
epochs = 100
our_bot = wb.PokerBot(maturity=epochs//4)

rounds_list = []

for i in trange(epochs):
  game = th.TexasHoldEm(buyin=500, big_blind=5, small_blind=2, max_players=2)
  game.start_hand()
  current_round = []
  while (game.is_hand_running()):
    # branch on whose turn
    if (game.current_player == 0):
      our_decision = smort_decision(our_bot, game)
      current_round.append(our_decision)
      if (our_decision[6].type == "FOLD"):
        who_won = 1
      else:
        who_won = -1
    else:
      who_won = baby_decision(game)
    if (who_won == -1 and len(game.board) == 5):
      jerry_hand_rank = eval.evaluate(game.get_hand(0), game.board)
      tom_hand_rank = eval.evaluate(game.get_hand(1), game.board)
      who_won = int(jerry_hand_rank >= tom_hand_rank)
  # now round is done, close round and go again
  won_or_lost_chips = game._get_last_pot().amount*((-1)**who_won)
  our_bot.log_round(won_or_lost_chips)
  rounds_list.append([current_round, won_or_lost_chips])


100%|██████████| 100/100 [00:31<00:00,  3.21it/s]


In [4]:
outcomes = np.zeros(len(rounds_list))

for i in range(len(rounds_list)):
  outcomes[i] = rounds_list[i][1]

print("Avg. outcomes:", np.mean(outcomes))
print("Avg. win rate:", np.mean(outcomes > 0))

Avg. outcomes: 9.51
Avg. win rate: 0.34
